# DB에서 상장 기업 정보를 받아 네이버 증권에서 주가 수집하기

강사님 ver.

In [3]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pymysql
import time

In [4]:
pymysql.install_as_MySQLdb()
load_dotenv(dotenv_path=".env_db")

True

In [5]:
# mysql에서 테이블 불러오기
engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
conn = engine.connect()

data = pd.read_sql('2024_07_29_stock_company_info', con=conn)
conn.close()

In [6]:
def str2int(x):
    x = int(x.replace(",", ""))
    return x

In [7]:
stock_price_detail = {}


for idx, code in enumerate(data['종목코드']):
    
    url = f"https://finance.naver.com/item/main.naver?code={code}" 
    r = requests.get(url)
    print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중", end="\r")
    soup = bs(r.text, 'lxml')
    
    
    # 가격
    price = int((soup.select_one(".today").text).strip("\n").split("\n")[1].replace(",", ""))
    # 변동금액
    price_chage = int((soup.select_one(".today").text).strip("\n").split("\n")[9].replace(",", ""))
    # 변화율
    rate_of_chage = float(((soup.select_one(".today").text).strip("\n").split("\n")[13]+(soup.select_one(".today").text).strip("\n").split("\n")[15]).replace("%",""))
    stock_price_detail.setdefault('현재가', []).append(price)
    stock_price_detail.setdefault('변동금액', []).append(price_chage)
    stock_price_detail.setdefault('변화율', []).append(rate_of_chage)
    
    
    table = soup.select_one(".no_info")
    
    for tr in table.select("tr"):
        for td in tr.select('td'):
            stock_price_detail.setdefault(td.select_one('span').text, []).append(str2int(td.select_one("span.blind").text))
    time.sleep(5)
    
    
    
stock_price_detail



KeyboardInterrupt: 

In [ ]:
data

# 수집과 동시에 DB에 저장하기 + 예외처리 하기

In [ ]:

print(today)

In [ ]:
today = str(date.today()).replace("-", "_")
print(today[:7])

In [ ]:
# DB에 접속해서 저장하는 함수


def stock_info_to_db(idx, code, df):
    from sqlalchemy import create_engine
    import pymysql
    from dotenv import load_dotenv
    from datetime import date
    pymysql.install_as_MySQLdb()
    load_dotenv(dotenv_path=".env_db")
    today = str(date.today()).replace("-","_")
    
    engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
    conn = engine.connect()
    df.to_sql(f"{today[:7]}_stock_price_info", con=conn, if_exists='append', index=False)
    conn.close()
    
    
    return print(f"{today}, {idx}, {code}, {'저장완료':<30s}", end="\r")
    
    
    

In [ ]:
errors = {}


for idx, (company, code) in enumerate(zip(data['회사명'], data['종목코드'])):
    stock_price_detail = {}
    # 반복문이 돌때마다 딕셔너리 만들어짐
    url = f"https://finance.naver.com/item/main.naver?code={code}" 
    
    try:
        r = requests.get(url)
        print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중                    ", end="\r")
        soup = bs(r.text, 'lxml')
        
        
        # 가격
        price = int((soup.select_one(".today").text).strip("\n").split("\n")[1].replace(",", ""))
        # 변동금액
        price_chage = int((soup.select_one(".today").text).strip("\n").split("\n")[9].replace(",", ""))
        # 변화율
        rate_of_chage = float(((soup.select_one(".today").text).strip("\n").split("\n")[13]+(soup.select_one(".today").text).strip("\n").split("\n")[15]).replace("%",""))
        
        
        stock_price_detail.setdefault('수집일', []).append(str(date.today()))
        stock_price_detail.setdefault('회사명', []).append(company)
        stock_price_detail.setdefault('종목코드', []).append(code)
        stock_price_detail.setdefault('현재가', []).append(price)
        stock_price_detail.setdefault('변동금액', []).append(price_chage)
        stock_price_detail.setdefault('변화율', []).append(rate_of_chage)
        
        
        table = soup.select_one(".no_info")
        
        
        
        for tr in table.select("tr"):
            for td in tr.select('td'):
                stock_price_detail.setdefault(td.select_one('span').text, []).append(str2int(td.select_one("span.blind").text))
        
        df = pd.DataFrame(stock_price_detail)
        # 바로 데이터프레임으로 만듦
        stock_info_to_db(idx, code, df)
        # db 함수 호출
        
        time.sleep(5)
        
    except Exception as e:
        print(e)
        print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중 에러", end="\r")
        errors.setdefault("에러", []).append(code)  # 에러난 코드들 모음
        
        
stock_price_detail


# try 로 데이터 수집하다가 에러가 뜨면 except 로 넘어감! 
